<a href="https://colab.research.google.com/github/veroorli/ProjetProg/blob/master/TME2_Jehanne_Reboud.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

BDLE 2022

date du document  :  07/10/2022

# TP2 et 3 Fenêtres


SQL avec fonctions sur des fenêtres

## Préparation

Lire le fichier des visites *sans* préciser le type des attributs.
Par défaut, tous les attributs sont considérés comme étant de type string.

In [ ]:
user_visits = spark.read.option("header", "True").option("delimiter", ";").format("csv").load(local_dir + "/" + "userVisits-Toro.csv")
user_visits.show(3)
user_visits.printSchema()

+----------+------------+----------+-----+---------+-------+-----+
|   photoID|      userID| dateTaken|poiID| poiTheme|poiFreq|seqID|
+----------+------------+----------+-----+---------+-------+-----+
|7941504100|10007579@N00|1346844688|   30|Structure|   1538|    1|
|4886005532|10012675@N05|1142731848|    6| Cultural|    986|    2|
|4886006468|10012675@N05|1142732248|    6| Cultural|    986|    2|
+----------+------------+----------+-----+---------+-------+-----+
only showing top 3 rows

root
 |-- photoID: string (nullable = true)
 |-- userID: string (nullable = true)
 |-- dateTaken: string (nullable = true)
 |-- poiID: string (nullable = true)
 |-- poiTheme: string (nullable = true)
 |-- poiFreq: string (nullable = true)
 |-- seqID: string (nullable = true)



Lire le fichier en précisant le schéma : nom et type des attributs

Nombre de photos, nombre de séquences et nombre d'évenements *un user a visité un POI* déterminés par un couple (séquence, POI)

In [ ]:
%%sql
select count(*) as nbPhotos,
 count(distinct seqID) 
as nbSequences, count(distinct seqID, poiID) 
as nb_visites
from user_visits

Les POI avec la plus grande frequence précalculée *poiFreq*

In [ ]:
%%sql
select distinct poiID, poiFreq 
from user_visits
order by poiFreq desc

Les POI avec la plus grande fréquence (ici la "fréquence bis" est le nombre de photos prises à un POI)

In [ ]:
%%sql
select poiID, max(poiFreq) as poiFreq, count(*) as poiFreqBis
from user_visits
group by poiID
order by poiFreqBis desc

### Les lieux visités : POI
Ils sont appelés *Point Of Interest*

In [ ]:
%%sql
cache table POI;

SELECT * 
FROM POI

la liste des thèmes

In [ ]:
%%sql
create or replace temp view themes as
select distinct theme
from POI
order by theme;

cache table themes;

select *
from themes

## Exercice 1

#### 1) Identifier les thèmes

Définir la table Theme1(theme_id, name) avec les numéros de thème commençant à 1 pour les noms de thème triés dans l'ordre croissant.

In [ ]:
%%sql

create or replace temp view Theme1 as
select distinct theme,rank() over (order by theme asc)
 as theme_id
from themes;

select * from Theme1

#### 2) Classement des séquences par leur plus grand nombre de POI **distincts**

Définir la table TopSeq(seqID, nbPOI, rang) avec rang=1 pour la séquence ayant le plus grand nbPOI.


On trie par séquence puis on compte le nombre de POIs distincts. Enfin on fait un classement des séquences en fonction de leur nombre de POIs distincts.

In [ ]:
%%sql

create or replace temp view TopSeq as
select seqID, count(distinct poiID) as nbPOI,
rank() over (order by count(distinct poiID) desc) as rang
from user_visits
group by seqID;

select * from TopSeq


#### 2) Identifier les check-ins




2a) Définir la table Visite1 telle qu'il n'y ait pas de doublons sur le triplet (seqID, poiID, date)


On compte les lignes ayant le même triplet seqID, poiID, date puis on filtre ce nombre de lignes à 1 pour supprimer les doublons.

In [ ]:
%%sql
create or replace temp view Temporaire as
select seqID, poiID, date, row_number()
 over (partition by seqID, poiID,
  date order by  seqID, poiID, date) as rang
from user_visits;




In [ ]:
%%sql
create or replace temp view Visite1 as
select seqID, poiID, date
from Temporaire
where rang <=1
order by seqID, date, poiID;


select *
from Visite1
where seqID in (298, 510)
order by seqID, date, poiID;


2b) Montrer qu'il existe des séquences où une même date est associée à plusieurs POI distincts. 
Indication : utiliser une requete de regroupement.

On trie par séquence et date puis on compte le nombre de POIs distincts dans chaque groupe. On séléctionne les groupes seq/date ayant plusieurs poi distincts.

In [ ]:

%%sql

create or replace temp view T as
select seqID, date, count(distinct poiID) as nb_POI_meme_date
from Visite1
group by seqID,date
order by seqID, date;




In [ ]:
%%sql

select *
from T
where nb_POI_meme_date >1
order by seqID, date;



2c) A partir de la table précédente, définir la table Visite2(seqID, poiID, date, num) avec *num* étant le numéro d'ordre dans une séquence.
Les POI visités à une même date sont triés dans l'ordre croissant de poiID.

Dans une même séquence on trie par date et par poi et on fait un classement .

In [ ]:
%%sql
create or replace temp view Visite2  as
select seqID,date,poiID,
rank() over (partition by seqID  order by  date, poiID) as num
from  Visite1;


select * from Visite2
where seqID in (298, 510)
order by seqID,num;

#### 3) Identifier les Visites de POI

Définir la table Visite3(userID, seqID, poiID, poiPosition) telle que la  poiPosition vaut *i* pour le ième POI visité dans une séquence.

Indications:

La visite d'un POI correspond à toutes les photos consécutives prises à ce POI.

Un même POI peut apparaitre **plusieurs** fois à des positions différentes dans une séquence si au moins un autre POI a été visité entre temps dans la séquence.


##### 3a) POI précédent
Commencer par associer chaque événement avec le POI précédent dans la séquence.
Indication penser aux fonctions lag() ou first()

Pour chaque séquence on trie par POI et date et on récupère avec la fonction lag() le POI à la date précédente.

In [ ]:
%%sql
create or replace temp view Visite3a as
select seqID,date,poiID, num, lag(poiID,1,poiID) over (partition by seqID order by seqID,date,poiID asc) as precedent
from  Visite2;

select * from Visite3a
where seqID in (298, 510)
order by seqID, date;


##### 3b) Début de visite
Ajouter un attribut *début* valant 1 pour le premier tuple d'une série de photos consécutives concernant le même POI et 0 sinon. 
Indication, penser à la syntaxe case when ...then ... else ... end

Lorsque le POI à la date précédente est le même qu'à la date courante début prend la valeur de 1.

In [ ]:
%%sql
create or replace temp view Visite3b as
select *,
case when poiID == precedent then 0
     else 1 end as debut
from Visite3a;

select * from Visite3b
where seqID in (298, 510)
order by seqID, date;

##### 3c) Ordonner les POI visités
Définir la table Visite3 décrite au début de la question 3). Ajouter l'attribut poiPosition 
 Visite3(userID, seqID, poiID, poiPosition)

Pour chaque séquence on trie par date et POI puis on somme début.

In [ ]:
%%sql

create or replace temp view Visite3 as
select seqID,num,date,poiID, sum(debut) over (partition by seqID order by date, poiID) as poiPosition
from Visite3b;

select * from Visite3
where seqID in (298, 510)
order by seqID, date, poiID;

#### 4) Durée de visite d'un POI
Définir la table Visite4(seqID, poiPosition, poiID, duree) comme étant la différence entre la plus grande et la plus petite date des photos consécutives associées à un même POI.

On trie les visites par sequence et poipostion,poiID puis pour chaque groupe on calcule la date maximale et la date minimale, ainsi que leur différence max(date)-min(date) pour obtenir la durée entre la première et la dernière photo d'une même séquence associée à un même POI. 

In [ ]:
%%sql

create or replace temp view Visite4 as
select seqID,poiPosition,poiID,
min(date) as date_debut, 
max(date) as date_fin,
max(date)-min(date) as duree
from Visite3
group by seqID,poiPosition,poiID ;


select * from Visite4
where seqID in (298, 510)
order by seqID,poiPosition;

verification: durée moyenne de visite d'un POI, pour les durée >0

In [ ]:
%%sql
select round(avg(duree)/60,1) as Moyenne_duree_en_minutes
from Visite4
where duree>0

#### 4a) nombre moyen de visites et nombre moyen de POI dans une séquence

On groupe par séquence puis on fait la moyenne du nombre de poiPosition et celle du nombre de poiID distincts.

In [ ]:
%%sql
create or replace temp view SeqNbVisite as
select seqID, count(poiPosition) as nbVisite,
 count(distinct poiID) as nbPOI
from Visite4
group by seqID;

select avg(nbVisite) as moyenne_nb_visite,
 avg(nbPOI) as moyenne_nb_POI
from SeqNbVisite

vérification fréquence des POI

In [ ]:
%%sql
select poiID, count(*) as poiFreqVisite
from Visite4
group by poiID
order by poiFreqVisite desc

#### 4b) Nombre de séquences selon leur nombre de visites. 

Afficher le nombre de séquences pour chaque nombre de visites existant.


In [ ]:
%%sql
create or replace temp view Visite4b as
select nbVisite, count(seqID) as nbSequences
from SeqNbVisite
group by nbVisite;

 Afficher aussi, pour chaque nombre de visite, le nombre *cumulé* de séquences ayant **au moins** ce nombre de visite.

Pour avoir le nombre cumulé des séquences ayant au moins ce nombre de visites on ordonne par nombre de visite et on fait une somme cumulée sur les nombre de séquence sur la ligne courante et toutes les prédédentes.

In [ ]:

%%sql
select nbVisite, nbSequences, 
sum(nbSequences) over (order by nbVisite 
desc rows between unbounded preceding and current row) 
as nbre_sequences_avec_au_moins_nbVisite
from Visite4b
order by nbVisite

Nombre de sequences selon leur nombre de POI distincts. 

Afficher aussi, pour chaque nombre de visite, le nombre cumulé de séquences ayant **au moins** ce nombre de POI distincts.

In [ ]:
%%sql
create or replace temp view Visite4bis as
select nbPOI, count(seqID) as nbSequences
from SeqNbVisite
group by nbPOI;

select nbPOI, nbSequences, 
sum(nbSequences) over 
(order by nbPOI desc rows between 
unbounded preceding and current row) 
as nbre_sequences_avec_au_moins_nbPOI
from Visite4bis
order by nbPOI

Nombre de séquences ayant au moins un POI visité 2 fois

In [ ]:
%%sql
select count(*) as nbSequences
from SeqNbVisite
where nbVisite > nbPOI


#### 5) Nombre de visites sur une semaine glissante
a) Définir la table Visite5a(userID, annee, mois, jour, nbVisite) : nbVisite est le nombre de visites qu'un utilisateur a fait chaque jour. 



On converti la date dans un format lisible puis on trie par user,année,mois,jour puis on compte les lignes pour le nombre de visites.

In [ ]:
%%sql
create or replace temp view Visite5a as
select userID, year(timestamp(date)) as annee,
month(timestamp(date))   as mois,
day(timestamp(date))   as jour ,
count(*) as nbVisite
from user_visits
group by userID, annee, mois, jour;

select * from Visite5a
order by userID, annee, mois, jour, nbVisite

b) En déduire la table Visite5b(userID, anne, mois, jour, nbVisite7jours) : nbVisite7jours étant le nombre de visites effectuées sur une semaine glissante.

Pour chaque user, on ordonne par date (annee,mois jour) puis on compte le nombre de visites sur une fenêtre glissante de 7 lignes précédent la courante soit les 7 jours précédents. 

In [ ]:
%%sql
select  distinct userID,
year(timestamp(date))  as annee,
month(timestamp(date)) as mois,
day(timestamp(date))   as jour,
count(timestamp(date)) over 
(partition by userID order by 
cast(from_unixtime(date) as date) 
range between 7 preceding and current row) 
as nbVisite7Jours
from user_visits

#### 6) Déplacements entre deux POI
Définir la table Duree_Deplacement(seqID, poiPosition, poiID, deplacement). 
*deplacement* est la durée depuis la fin de la visite du POI courant jusqu'au début de visite du prochain POI dans une séquence.


On trie par séquence et POI. Puis on calcule la différence entre la date du POI courant (la ligne courante) et date de la ligne suivante, soit la date de visite du prochain POI.

In [ ]:
%%sql
create or replace temp view Duree_Deplacement as
select seqID,poiPosition,poiID,
max(timestamp(date)) over 
(partition by seqID, poiPosition 
rows between current row and 1 following) 
- timestamp(date) as deplacement
from visite3;

select * from Duree_Deplacement
where seqID in (298, 510)
order by seqID, poiPosition, poiID;

## Exercice 2 : YFCC

#### Données YFCC France

#### Question 1

Extraire de YFCC les sequences de points des utilisateurs telles que les conditions suivantes soient vérifiées :

Une séquence ne peut pas couvrir plusieurs jours. Si un utilisateur a pris des photos pendant plusieurs jours consécutifs, cela forme plusieurs séquences.

Une séquence doit contenir au moins 3 points distincts.

Chaque point doit être associé à au moins 3 utilisateurs.

On peut supposer que deux photos prises à deux positions GPS très proches (moins de *d* mètres entre les deux positions) correspondent à un même point. 


In [ ]:
yfcc_france.createOrReplaceTempView("yfcc_france")

Chaque point doit être associé à au moins 3 utilisateurs: 

1) On identifie les POIs avec un poiID en fonction de leur position latitude et longitude.

2)On regroupe par POI donc (longitude, latitude) et on compte les users pour chaque POI.

3)On filtre les POI ayant plus de 2 users.

In [ ]:
%%sql

create or replace temp view T0 as
select poi_id, Longitude, Latitude
from (
  select Longitude, Latitude,
  row_number() over (order by Longitude, Latitude) as poi_id,
  count(UserNSID) as nb_users
  from yfcc_france
  group by Longitude, Latitude
)
where nb_users > 2;

select *
from T0

4) On fait la jointure sur les POI_id ( latitudes, longitudes) du tableau général et de la selection précédente des POIs ayant au moins 3 users pour récupérer l'ensemble de leur caractéristiques.

In [ ]:
%%sql
create or replace temp view T1 as
select distinct y.UserNSID, y.DateTaken, y.Longitude, y.Latitude, t0.poi_id
from  yfcc_france y, T0 t0
where y.Latitude == t0.Latitude and y.Longitude == t0.Longitude ;

select *
from T1

Une séquence doit contenir au moins 3 points distincts: \\

1)on trie par user et par date 

2)on fait une liste des POIs distincts à chaque séquence

3)on filtre les séquences qui ont une liste de pois distincts supérieurs à 2

In [ ]:
%%sql

create or replace temp view T2 as
select *
from (
  select UserNSID, DateTaken, collect_list(distinct poi_id) as liste_POI
  from T1
  group by UserNSID, DateTaken
  order by UserNSID, DateTaken
)
where cardinality(liste_POI) > 2;

select *
from T2

#### Question 2

Proposer un exemple d'analyse des séquences obtenues à la question précédente suivante 3 dimensions dont au moins une doit avoir au moins 3 niveaux.